In [1]:
using SparseArrays, LinearAlgebra

function readgraph(graph)
    I=Int[]
    J=Int[]
    open(graph) do f
        linecounter = 0
        for l in eachline(f)
            linecounter += 1
            v = split(l)
            if v[1] == "D"
                i,j = extrema((parse(Int, v[2])+1, parse(Int, v[3])+1))
                push!(I, i)
                push!(J, j)
            end
        end
    end
    N = maximum(I)
    H = sparse(J.-N,I,fill(1,length(I))) .÷ 2
end


function readseeds(seeds, H)
    M, N = size(H)
    @show size(H)
    v = Int[]
    open(seeds) do f
        for l in eachline(f)
            if l[1] != '#'
                v = parse.(Int, split(l)) .- N .+ 1
            end
        end
        println("$(length(v)) removed factors $v")
        return H[setdiff(1:size(H,1), v), :]
    end
end

readseeds (generic function with 1 method)

In [2]:
function leaf_removal(H::SparseMatrixCSC)
    M, N = size(H)
    Ht = sparse(transpose(H)) # for faster access by row
    degs = vec(sum(H .!= 0, dims=1))
    facts = fill(true, M)
    rowperm = Int[]
    Q = findall(degs .== 1)
    indep = findall(degs .== 0)
    dep = Int[]
    while !isempty(Q)
        i = popfirst!(Q)
        degs[i] == 0 && continue
        push!(dep, i)
        ∂i = rowvals(H)[nzrange(H,i)]
        ∂i = ∂i[facts[∂i]]
        @assert length(∂i) == 1 # should be a residual leaf
        a = ∂i[1]
        facts[a] = false
        push!(rowperm, a) 
        for j in rowvals(Ht)[nzrange(Ht,a)]
            degs[j] -= 1
            if j != i
                if degs[j] == 0
                    push!(indep, j)
                elseif degs[j] == 1
                    push!(Q, j)
                end
            end
        end
    end
    all(degs .==  0) || @warn "non-empty core"
    rowperm, [dep; indep]
end

leaf_removal (generic function with 1 method)

In [3]:
isuppertriang(H::SparseMatrixCSC) = all(rowvals(H)[last(nzrange(H,i))] == i for i = 1:size(H,1))

function ut2diagGF2!(T::SparseMatrixCSC)
    (m,n) = size(T)
    # Check that the left part of T is unit upper triangular
    @assert isuppertriang(T)
    # Loop over diagonal elements
    for c in m:-1:2
        # Find non-zero elements above T[c,c] and perform row operations to 
        #  cancel them out
        for (j,v) in @views zip(rowvals(T[:,c]),nonzeros(T[:,c]))
            if v != 0 && j < c
                for k in rowvals(T[c,c:end]).+c.-1
                    T[j,k] ⊻= T[c,k]
                end
            end
        end
    end
    dropzeros!(T)
end

ut2diagGF2! (generic function with 1 method)

In [4]:
function findbasis(H)
    rowperm, colperm = leaf_removal(H)
    Hnew = H[rowperm, colperm]
    ut2diagGF2!(Hnew)
    B = [Hnew[:, size(Hnew, 1)+1:end]; I]
    indep = colperm[size(H,1)+1:end]
    B[invperm(colperm),:], indep
end 

findbasis (generic function with 1 method)

In [141]:
H = readgraph("/home/ab/src/spread/graphN9000.txt")
H = readseeds("/home/ab/src/spread/seedsN9000.txt", H)
B, indep = findbasis(H);
M,N = size(H)
@show R=1-M/N

size(H) = (6600, 9000)
5 removed factors [129, 1515, 1531, 1655, 3471]
R = 1 - M / N = 0.26722222222222225


0.26722222222222225

In [142]:
rowperm,colperm = leaf_removal(H)

([4568, 97, 303, 4962, 589, 3783, 2642, 554, 3162, 1042  …  1081, 4040, 5522, 198, 5901, 3075, 7, 3606, 4236, 3265], [91, 353, 364, 685, 3356, 3896, 4261, 4784, 4976, 5077  …  5628, 7363, 4170, 8751, 4673, 5935, 798, 7435, 4491, 8359])

In [143]:
H[rowperm, colperm]

6595×9000 SparseMatrixCSC{Int64, Int64} with 19783 stored entries:
⠙⢮⣖⣂⡇⡐⠂⣆⢥⣰⣤⣚⡴⠗⢭⢰⣝⠰⣷⠢⡩⣓⣂⢎⢓⡧⣵⢷⡆⣿⢍⡦⣘⣀⣧⢄⡲⢦⡲⠐
⠀⠀⠙⢿⡿⣝⢲⢮⢮⣧⢛⣮⡧⡷⣽⡀⡡⠮⡒⠮⣜⣓⠛⡹⡿⠕⢯⢭⠳⣿⡐⣡⡾⠟⠯⢓⢈⡢⠽⢆
⠀⠀⠀⠀⠙⢮⡙⢿⣻⠕⢙⣼⠪⣨⡞⡙⣴⡳⣹⡻⡢⡾⢚⢁⡵⡿⢸⡗⢫⣧⠶⡅⠏⣇⡦⣌⠳⠹⠐⣨
⠀⠀⠀⠀⠀⠀⠙⢦⡈⠛⢯⣧⠫⠹⢳⡻⢏⣓⠴⣋⣼⣗⣤⣶⡘⢧⣩⡭⠾⣿⢊⠹⣽⡻⠆⢣⠹⠦⡶⣔
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠻⢷⡗⣟⣆⡓⣧⢵⡓⣎⠋⡬⣭⡰⡥⡥⣼⣹⣷⠐⢘⣫⣟⡊⣰⣃⠿⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠙⠿⣮⠽⡿⠼⢌⢈⣇⡓⢹⡻⣷⠹⡻⠺⣿⡵⣫⣗⣣⡱⡳⢧⢑⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠳⣕⣹⣵⣍⠹⣨⢷⢱⡀⣣⠧⠀⣿⣿⣯⠵⣡⣾⡵⢔⠷⡶
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢮⣾⣻⠙⡾⣶⢧⡮⣨⡂⠸⣿⣿⣹⣋⣯⣻⣍⣨⠽
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢷⡜⣩⠆⡹⡼⠆⠀⠀⢿⣿⣧⡳⣵⡬⢍⠍⡛
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢶⡯⠿⣺⣐⡁⠀⠈⣿⣿⣿⣻⠗⠶⣽⠭
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⣧⡻⠷⠂⠀⠀⠘⣿⣿⣿⣧⣏⡦⢟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⢷⢗⡀⠀⠀⠀⠘⢿⣿⣿⣮⢷⡟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⢧⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⠋
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡘⡇⠀⠀⠀⠀⠀⠀⠘⠿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠃⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [216]:
include("../code/sp.jl")

iteration! (generic function with 1 method)

In [217]:
using StaticArrays, OffsetArrays
x0 = rand((0,1), size(H,2))
s = (-1).^x0
J = 20
init=OffsetArray(MVector{2J+1}(fill(1/(2J+1),2J+1)), -J:J)
sp = survey_propagation(H; field=s, init, y=2.0);

In [178]:
using ProgressMeter

callback(t, ε, sp) = (println("$t $ε"); false)

function decimate!(sp; decsteps=10)
    H = sp.H
    freevars = copy(indep)
    iteration!(sp, maxiter=500, tol=1e-5, callback=callback, damp=0.5)
    while !isempty(freevars)
        iteration!(sp, maxiter=100, tol=1e-3, damp=0.5)
        mag = map(p->p'eachindex(p), sp.survey)
        sort!(freevars, by=i->abs(mag[i]))
        freevars, tofix = freevars[1:end-decsteps-1], freevars[max(1,end-decsteps):end]
        println("fixing: $tofix")
        sp.efield[tofix] .= sp.J.*sign.(mag[tofix])
        x = Vector(mag .< 0)
        x1 = (B*x[indep]) .% 2
        println("$(sum(H*x .%2)) $(sum(x .!= x0)/length(x)) $(sum(x1 .!= x0)/length(x)) $(length(freevars)/length(indep))")
    end
end

decimate!(sp; decsteps=50)

1 0.975609756097561
2 0.4878048780487805
3 0.4878048780487803
4 0.4103771053725435
5 0.3979789822674914
6 0.32122361225636187
7 0.2802173771537033
8 0.30303194918107407
9 0.3014512407972626
10 0.2847234467561418
11 0.29325163017431644
12 0.2869254836337214
13 0.2769288230253195
14 0.269262472079106
15 0.25676840649168475
16 0.25227479754015913
17 0.2577367587027869
18 0.2602962717859349
19 0.2636611653147879
20 0.27700218511400493
21 0.2709667759298026
22 0.25801638517768677
23 0.2576009397741854
24 0.2507474417967208
25 0.25857076142529
26 0.2677538659752566
27 0.26093201709790725
28 0.24493796251968852
29 0.2432361193133903
30 0.2357328734189541
31 0.22274000046528292
32 0.20517252256222585
33 0.19236122984468362
34 0.18117152643329404
35 0.17932238625239338
36 0.17237861634823592
37 0.16648075159261638
38 0.15894791558616161
39 0.14522433018004244
40 0.14105378236330934
41 0.13473946538628068
42 0.12364331547219287
43 0.10931333638883234
44 0.098995447528954
45 0.09771057477196304
4

In [219]:
function callback(t, ε, sp)
    t % 10 != 0 && return false
    mag = map(p->p'sign.(eachindex(p)), sp.survey)
    x = Vector(mag .< 0)
    x1 = (B*x[indep]) .% 2
    check = sum(H*x .%2)
    println("$t $check $(sum(x .!= x0)/length(x)) $(sum(x1 .!= x0)/length(x))")
    return check == 0
end
iteration!(sp, maxiter=100000, tol=1e-5, damp=0.2, rein=0.1, callback=callback)

10 281 0.202 0.2833333333333333
20 262 0.20477777777777778 0.27844444444444444
30 236 0.20666666666666667 0.2713333333333333
40 239 0.207 0.27444444444444444
50 215 0.20966666666666667 0.27155555555555555
60 207 0.21 0.27044444444444443
70 199 0.21166666666666667 0.2728888888888889
80 188 0.21244444444444444 0.2693333333333333
90 167 0.21266666666666667 0.27055555555555555
100 170 0.21222222222222223 0.27255555555555555
110 165 0.2127777777777778 0.2688888888888889
120 160 0.2137777777777778 0.2703333333333333
130 142 0.21555555555555556 0.26944444444444443
140 140 0.21544444444444444 0.2653333333333333
150 133 0.21644444444444444 0.2657777777777778
160 133 0.21755555555555556 0.2598888888888889
170 111 0.219 0.259
180 97 0.22022222222222224 0.25733333333333336
190 77 0.2212222222222222 0.25266666666666665
200 77 0.22155555555555556 0.24988888888888888
210 62 0.22233333333333333 0.247
220 73 0.222 0.24855555555555556
230 72 0.22211111111111112 0.2511111111111111
240 67 0.223 0.25111111

In [206]:
mag = map(p->p'eachindex(p), sp.survey);
x = Vector(mag .< 0)
xbase = B*(x[indep]) .% 2;

In [207]:
all(sign.(mag[indep]) == sign.(sp.efield[indep])) 

false

In [208]:
@show sum(x .!= x0)/length(xbase) 
@show sum(H*x .% 2)
@show sum(xbase .!= x0)/length(xbase)
@show sum(H*xbase .%2)/size(H,1)

sum(x .!= x0) / length(xbase) = 0.2058888888888889
sum((H * x) .% 2) = 235
sum(xbase .!= x0) / length(xbase) = 0.26766666666666666
sum((H * xbase) .% 2) / size(H, 1) = 0.0


0.0

In [209]:
sum(p[begin] for p in sp.survey)

2.2417460331495522e-69

In [128]:
[p[[-sp.J,sp.J]] for p in sp.survey[indep]]

2405-element Vector{Vector{Float64}}:
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [1.0, 0.0]
 [1.0, 0.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [1.0, 0.0]
 [0.0, 1.0]
 ⋮
 [1.0, 0.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [0.0, 1.0]

In [124]:
[(1,2)...]

2-element Vector{Int64}:
 1
 2

In [27]:
using ProgressMeter
callback(t,fg) = nothing;#println("$t $(fg.belief[1:4])")

nsamples = 10
fg = maxsum(H,  1 .- 2y .+ 1e-8*randn(N));

# the mapping is (0,1)=>(1,-1)
dist = map(1:nsamples) do i
    y = rand((0,1), N)
    fg.M .= 0
    fg.efield .= 1 .- 2y .+ 1e-8*randn(N)
    iteration!(fg; maxiter=2000, γ=1e-3, callback=callback)
    x = Int.(fg.belief .< 0)
    c = sum(fg.H'x .% 2)
    d = sum(x .⊻ y)/N
    @show d c
    x1 = B*x[indep] .% 2; d1 = sum(x1 .⊻ y)/N; c1 = sum(fg.H'x1 .% 2); @show d1 c1 
    d1
end
sum(dist)/nsamples

LoadError: UndefVarError: y not defined

In [28]:
1-M/N,sum(dist)/nsamples

LoadError: UndefVarError: dist not defined

In [29]:
x=[1:10;]; f=(x .% 2 .== 1);x[findall(f)]==x[f]

true

In [52]:
dist = map(1:10) do i
    Random.seed!(i)
    
    y = rand((0,1), N)

    lm = LossyModelGF2(fg, zeros(N), Inf, 1.0, y)
    
    function checker()
        v = (B*((fg.fields .< 0)[indep])) .% 2;
        dist = sum(v .⊻ y)/N
        check = sum(H*(fg.fields .< 0) .% 2)
        check, dist
    end

    independent = falses(N)
    independent[indep] .= true
    res_fg = solve!(lm, fg, randseed=1, verbose=true, showprogress=true, 
        independent=independent, basis = B)
#    res_fg = solve!(lm, fg, randseed=1, verbose=true, showprogress=true)
    avg_dist = res_fg.distortion 
    check, dist = checker()
    @show dist check avg_dist
    dist
end

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24287749287749288
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24786324786324787
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


...

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.245014245014245
check = 6
avg_dist = 0.4458689458689459
dist = 0.24715099715099714
check = 0
avg_dist = 0.24715099715099714


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.25427350427350426
check = 2
avg_dist = 0.44515669515669515
dist = 0.24857549857549857
check = 0
avg_dist = 0.24857549857549857


10-element Vector{Float64}:
 0.24287749287749288
 0.24786324786324787
 0.25427350427350426
 0.26495726495726496
 0.2378917378917379
 0.26851851851851855
 0.245014245014245
 0.24715099715099714
 0.25427350427350426
 0.24857549857549857

In [53]:
dev = sqrt(sum(dist.^2)/length(dist)-(sum(dist)/length(dist))^2)/sqrt(length(dist))
1-M/N, sum(dist)/length(dist)

(0.24002849002849003, 0.2511396011396011)